In [1]:
import pandas as pd
import numpy as np
import os,json

In [2]:
# Функция для сбора и чтения циклом файлов

def json_opener(path):
    path_to_file = path
    list = []
    for i in [i for i in os.listdir(path_to_file)]:
        list.append(path_to_file + i)
    df = pd.concat((pd.read_json(f) for f in list))
    return df

In [3]:
# читаем sleep data и создаем датафрейм

sleep = json_opener('Sleep Data/')

In [4]:
# Выводим первые 5 строк
sleep.head()

,sleepStartTimestampGMT,sleepEndTimestampGMT,calendarDate,sleepWindowConfirmationType,retro,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,unmeasurableSeconds
0,2020-12-31T20:00:00.0,2021-01-01T05:00:00.0,2021-01-01,UNCONFIRMED,False,NaN,NaN,NaN,NaN,NaN
1,2021-01-01T23:51:00.0,2021-01-02T08:01:00.0,2021-01-02,ENHANCED_CONFIRMED_FINAL,False,3300.0,20520.0,5460.0,120.0,0.0
2,2021-01-03T00:34:00.0,2021-01-03T09:00:00.0,2021-01-03,ENHANCED_CONFIRMED_FINAL,False,3900.0,18000.0,8340.0,120.0,0.0
3,2021-01-04T00:33:00.0,2021-01-04T07:44:00.0,2021-01-04,ENHANCED_CONFIRMED_FINAL,False,0.0,11880.0,13920.0,60.0,0.0
4,2021-01-04T23:31:00.0,2021-01-05T09:04:00.0,2021-01-05,ENHANCED_CONFIRMED_FINAL,False,6300.0,16200.0,11820.0,60.0,0.0


In [5]:
# Знакомимся с данными
sleep.info()

<class 'pandas.core.frame.DataFrame'>
Index: 781 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sleepStartTimestampGMT       781 non-null    object 
 1   sleepEndTimestampGMT         781 non-null    object 
 2   calendarDate                 781 non-null    object 
 3   sleepWindowConfirmationType  781 non-null    object 
 4   retro                        781 non-null    bool   
 5   deepSleepSeconds             756 non-null    float64
 6   lightSleepSeconds            756 non-null    float64
 7   remSleepSeconds              738 non-null    float64
 8   awakeSleepSeconds            756 non-null    float64
 9   unmeasurableSeconds          756 non-null    float64
dtypes: bool(1), float64(5), object(4)
memory usage: 61.8+ KB


In [6]:
# Преабразуем строки в datetime

sleep['sleepStartTimestampGMT'] = pd.to_datetime(sleep['sleepStartTimestampGMT'])
sleep['sleepEndTimestampGMT'] = pd.to_datetime(sleep['sleepEndTimestampGMT'])
sleep['calendarDate'] = pd.to_datetime(sleep['calendarDate'])

In [7]:
# Переводим время GMT в локальное

sleep['sleepStartTimestampGMT'] = sleep['sleepStartTimestampGMT'] + pd.Timedelta(hours=3)
sleep['sleepEndTimestampGMT'] = sleep['sleepEndTimestampGMT'] + pd.Timedelta(hours=3)

In [8]:
# Проверяем наличие дубликатов
sleep[sleep.duplicated()]

,sleepStartTimestampGMT,sleepEndTimestampGMT,calendarDate,sleepWindowConfirmationType,retro,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,unmeasurableSeconds


In [9]:
# Убираем столбцы которые не несут информации для анализа

sleep.drop(['sleepWindowConfirmationType', 'retro'], axis=1, inplace=True)

In [10]:
# Удаляем пропуски

sleep.dropna(inplace=True)

In [11]:
# Добавляем колонку, суммирующую секунды сна

sleep['totalSleep'] = sleep['deepSleepSeconds'] + sleep['lightSleepSeconds'] + sleep['remSleepSeconds'] + sleep['awakeSleepSeconds'] + sleep['unmeasurableSeconds']

In [12]:
sleep.head()

,sleepStartTimestampGMT,sleepEndTimestampGMT,calendarDate,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,unmeasurableSeconds,totalSleep
1,2021-01-02 02:51:00,2021-01-02 11:01:00,2021-01-02,3300.0,20520.0,5460.0,120.0,0.0,29400.0
2,2021-01-03 03:34:00,2021-01-03 12:00:00,2021-01-03,3900.0,18000.0,8340.0,120.0,0.0,30360.0
3,2021-01-04 03:33:00,2021-01-04 10:44:00,2021-01-04,0.0,11880.0,13920.0,60.0,0.0,25860.0
4,2021-01-05 02:31:00,2021-01-05 12:04:00,2021-01-05,6300.0,16200.0,11820.0,60.0,0.0,34380.0
5,2021-01-06 04:14:00,2021-01-06 11:53:00,2021-01-06,1380.0,19140.0,7020.0,0.0,0.0,27540.0


In [13]:
# Читаем файлы с инфо об активности

In [14]:
list = []
for i in [i for i in os.listdir('Activity Data/')]:
    list.append('Activity Data/' + i)
list

['Activity Data/UDSFile_2020-12-13_2021-03-23.json',
 'Activity Data/UDSFile_2021-03-23_2021-07-01.json',
 'Activity Data/UDSFile_2021-07-01_2021-10-09.json',
 'Activity Data/UDSFile_2021-10-09_2022-01-17.json',
 'Activity Data/UDSFile_2022-01-17_2022-04-27.json',
 'Activity Data/UDSFile_2022-04-27_2022-08-05.json',
 'Activity Data/UDSFile_2022-08-05_2022-11-13.json',
 'Activity Data/UDSFile_2022-11-13_2023-02-21.json']

In [15]:
# Проверка распаковки (видим что есть несколько уровней заголовков)

json_opener('Activity Data/').head(3)

,userProfilePK,calendarDate,uuid,durationInMilliseconds,totalKilocalories,activeKilocalories,bmrKilocalories,wellnessKilocalories,remainingKilocalories,wellnessTotalKilocalories,...,includesSleepPulseOx,source,allDayStress,bodyBattery,minAvgHeartRate,maxAvgHeartRate,version,hydration,isDailyTotalFromEpochData,restingCaloriesFromActivity
0,92343958,"{'date': 'Jan 1, 2021 12:00:00 AM'}",1c93012525624c80ac8d315ed7588bdd,86400000,1560,72,1488,1560,1560,1560,...,False,0,"{'userProfilePK': 92343958, 'calendarDate': {'...","{'userProfilePK': 92343958, 'calendarDate': {'...",61.0,104.0,46980002,NaN,NaN,NaN
1,92343958,"{'date': 'Jan 2, 2021 12:00:00 AM'}",98d563849aa74777a691e93c64d26419,86400000,1762,274,1488,1762,1762,1762,...,False,0,"{'userProfilePK': 92343958, 'calendarDate': {'...","{'userProfilePK': 92343958, 'calendarDate': {'...",58.0,120.0,73020003,NaN,NaN,NaN
2,92343958,"{'date': 'Jan 3, 2021 12:00:00 AM'}",d124be96dfbb437bb532273e75f00b9a,86400000,1704,216,1488,1704,1704,1704,...,False,0,"{'userProfilePK': 92343958, 'calendarDate': {'...","{'userProfilePK': 92343958, 'calendarDate': {'...",59.0,120.0,46260003,NaN,NaN,NaN


In [16]:
# Собираем датафрейм через json_normalize
path = 'Activity Data/'
list = []
for i in [i for i in os.listdir(path)]:
    list.append(path + i)
data=[]
for i in list:
    data.append(pd.json_normalize(json.load(open(i))))
    activity = pd.concat(data)

In [17]:
# Не распаковались только столбцы allDayStress.aggregatorList и bodyBattery.bodyBatteryStatList. С первым будем работать отдельно, второй не нужен
activity.head()

,userProfilePK,uuid,durationInMilliseconds,totalKilocalories,activeKilocalories,bmrKilocalories,wellnessKilocalories,remainingKilocalories,wellnessTotalKilocalories,wellnessActiveKilocalories,...,bodyBattery.chargedValue,bodyBattery.drainedValue,bodyBattery.bodyBatteryVersion,bodyBattery.bodyBatteryStatList,hydration.userProfilePK,hydration.calendarDate.date,hydration.activityIntakeInML,hydration.capped,isDailyTotalFromEpochData,restingCaloriesFromActivity
0,92343958,1c93012525624c80ac8d315ed7588bdd,86400000,1560.0,72.0,1488.0,1560.0,1560.0,1560.0,72.0,...,23.0,14.0,1.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",NaN,NaN,NaN,NaN,NaN,NaN
1,92343958,98d563849aa74777a691e93c64d26419,86400000,1762.0,274.0,1488.0,1762.0,1762.0,1762.0,274.0,...,46.0,73.0,1.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",NaN,NaN,NaN,NaN,NaN,NaN
2,92343958,d124be96dfbb437bb532273e75f00b9a,86400000,1704.0,216.0,1488.0,1704.0,1704.0,1704.0,216.0,...,34.0,33.0,1.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",NaN,NaN,NaN,NaN,NaN,NaN
3,92343958,b6da4f578c1e49909535374ab3578d57,86400000,1628.0,140.0,1488.0,1628.0,1628.0,1628.0,140.0,...,77.0,41.0,1.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",92343958.0,"Jan 4, 2021 12:00:00 AM",0.0,False,NaN,NaN
4,92343958,c8893ad3ebac4137a52e2866280f2751,86400000,1571.0,83.0,1488.0,1571.0,1571.0,1571.0,83.0,...,64.0,47.0,1.0,"[{'bodyBatteryStatType': 'HIGHEST', 'statsValu...",NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# В allDayStress.aggregatorList интересует только раздел AWAKE. Дважды применяем json_normalize для распаковки интересующего столбца

allDayStress = pd.json_normalize(pd.json_normalize(activity['allDayStress.aggregatorList'])[1])
allDayStress.head()

,type,averageStressLevel,averageStressLevelIntensity,maxStressLevel,stressIntensityCount,stressOffWristCount,stressTooActiveCount,totalStressCount,totalStressIntensity,stressDuration,restDuration,activityDuration,uncategorizedDuration,totalDuration,lowDuration,mediumDuration,highDuration
0,AWAKE,31,23,91.0,323.0,303.0,48,674,2871.0,8760.0,10620.0,2880,18180.0,40440,4500.0,3480.0,780.0
1,AWAKE,52,49,94.0,693.0,67.0,185,945,-22282.0,36540.0,5040.0,11100,4020.0,56700,13980.0,16740.0,5820.0
2,AWAKE,50,49,93.0,733.0,86.0,113,932,-23584.0,39060.0,4920.0,6780,5160.0,55920,17220.0,18540.0,3300.0
3,AWAKE,33,27,90.0,738.0,151.0,115,1004,-2498.0,23100.0,21180.0,6900,9060.0,60240,16020.0,5700.0,1380.0
4,AWAKE,30,24,88.0,631.0,154.0,81,866,3945.0,16140.0,21720.0,4860,9240.0,51960,9240.0,6240.0,660.0


In [19]:
# добавляем столбцы из allDayStress в activity

for i in allDayStress:
    name = 'allDayStress.'+i
    activity[name] = allDayStress[i]

In [20]:
activity.head()

,userProfilePK,uuid,durationInMilliseconds,totalKilocalories,activeKilocalories,bmrKilocalories,wellnessKilocalories,remainingKilocalories,wellnessTotalKilocalories,wellnessActiveKilocalories,...,allDayStress.totalStressCount,allDayStress.totalStressIntensity,allDayStress.stressDuration,allDayStress.restDuration,allDayStress.activityDuration,allDayStress.uncategorizedDuration,allDayStress.totalDuration,allDayStress.lowDuration,allDayStress.mediumDuration,allDayStress.highDuration
0,92343958,1c93012525624c80ac8d315ed7588bdd,86400000,1560.0,72.0,1488.0,1560.0,1560.0,1560.0,72.0,...,674,2871.0,8760.0,10620.0,2880,18180.0,40440,4500.0,3480.0,780.0
1,92343958,98d563849aa74777a691e93c64d26419,86400000,1762.0,274.0,1488.0,1762.0,1762.0,1762.0,274.0,...,945,-22282.0,36540.0,5040.0,11100,4020.0,56700,13980.0,16740.0,5820.0
2,92343958,d124be96dfbb437bb532273e75f00b9a,86400000,1704.0,216.0,1488.0,1704.0,1704.0,1704.0,216.0,...,932,-23584.0,39060.0,4920.0,6780,5160.0,55920,17220.0,18540.0,3300.0
3,92343958,b6da4f578c1e49909535374ab3578d57,86400000,1628.0,140.0,1488.0,1628.0,1628.0,1628.0,140.0,...,1004,-2498.0,23100.0,21180.0,6900,9060.0,60240,16020.0,5700.0,1380.0
4,92343958,c8893ad3ebac4137a52e2866280f2751,86400000,1571.0,83.0,1488.0,1571.0,1571.0,1571.0,83.0,...,866,3945.0,16140.0,21720.0,4860,9240.0,51960,9240.0,6240.0,660.0


In [21]:
# Сохраняем дип копи для возможности вернуться к добытым с таим трудом данным :)

raw_activity = activity.copy(deep=True)

In [22]:
# Оставляем только интересующие столбцы

activity = activity[['calendarDate.date','totalKilocalories','activeKilocalories','totalSteps','highlyActiveSeconds','activeSeconds','currentDayRestingHeartRate',
          'bodyBattery.chargedValue','bodyBattery.drainedValue', 'allDayStress.averageStressLevel','allDayStress.averageStressLevelIntensity',\
          'allDayStress.totalDuration', 'allDayStress.lowDuration', 'allDayStress.mediumDuration', 'allDayStress.highDuration']]

In [23]:
# проверяем тип данных

activity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 781 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   calendarDate.date                         781 non-null    object 
 1   totalKilocalories                         781 non-null    float64
 2   activeKilocalories                        781 non-null    float64
 3   totalSteps                                781 non-null    int64  
 4   highlyActiveSeconds                       781 non-null    int64  
 5   activeSeconds                             781 non-null    int64  
 6   currentDayRestingHeartRate                781 non-null    int64  
 7   bodyBattery.chargedValue                  780 non-null    float64
 8   bodyBattery.drainedValue                  780 non-null    float64
 9   allDayStress.averageStressLevel           781 non-null    int64  
 10  allDayStress.averageStressLevelIntensity  78

In [24]:
# Преобразовываем дату в datetime

activity.loc[:,'calendarDate'] = pd.to_datetime(activity['calendarDate.date'], format="%b %d, %Y %H:%M:%S %p").dt.floor('d')

# удаляем не актуальный столбец
activity.drop('calendarDate.date', axis=1, inplace=True)

In [25]:
# Избавляемся от дубликатов

activity.drop_duplicates(inplace=True)

In [26]:
# Собираем итоговый датафрейм
data = pd.merge(sleep, activity, on='calendarDate', how='left')
data

,sleepStartTimestampGMT,sleepEndTimestampGMT,calendarDate,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,unmeasurableSeconds,totalSleep,totalKilocalories,...,activeSeconds,currentDayRestingHeartRate,bodyBattery.chargedValue,bodyBattery.drainedValue,allDayStress.averageStressLevel,allDayStress.averageStressLevelIntensity,allDayStress.totalDuration,allDayStress.lowDuration,allDayStress.mediumDuration,allDayStress.highDuration
0,2021-01-02 02:51:00,2021-01-02 11:01:00,2021-01-02,3300.0,20520.0,5460.0,120.0,0.0,29400.0,1762.0,...,5995,64,46.0,73.0,52,49,56700,13980.0,16740.0,5820.0
1,2021-01-03 03:34:00,2021-01-03 12:00:00,2021-01-03,3900.0,18000.0,8340.0,120.0,0.0,30360.0,1704.0,...,4325,64,34.0,33.0,50,49,55920,17220.0,18540.0,3300.0
2,2021-01-04 03:33:00,2021-01-04 10:44:00,2021-01-04,0.0,11880.0,13920.0,60.0,0.0,25860.0,1628.0,...,4544,66,77.0,41.0,33,27,60240,16020.0,5700.0,1380.0
3,2021-01-05 02:31:00,2021-01-05 12:04:00,2021-01-05,6300.0,16200.0,11820.0,60.0,0.0,34380.0,1571.0,...,2575,66,64.0,47.0,30,24,51960,9240.0,6240.0,660.0
4,2021-01-06 04:14:00,2021-01-06 11:53:00,2021-01-06,1380.0,19140.0,7020.0,0.0,0.0,27540.0,1910.0,...,6908,65,42.0,70.0,35,28,58680,9180.0,7800.0,1440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2023-02-15 00:28:00,2023-02-15 09:24:00,2023-02-15,6600.0,19200.0,5640.0,720.0,0.0,32160.0,1924.0,...,7875,53,27.0,74.0,39,34,59820,17280.0,12120.0,2940.0
734,2023-02-16 00:05:00,2023-02-16 08:55:00,2023-02-16,2700.0,19140.0,9900.0,60.0,0.0,31800.0,1800.0,...,4380,61,96.0,63.0,38,31,59940,14400.0,11760.0,1860.0
735,2023-02-16 22:29:00,2023-02-17 08:42:00,2023-02-17,0.0,26640.0,8880.0,1260.0,0.0,36780.0,1846.0,...,4265,57,30.0,62.0,47,43,59220,18660.0,13800.0,3540.0
736,2023-02-18 00:10:00,2023-02-18 09:30:00,2023-02-18,4080.0,22620.0,6900.0,0.0,0.0,33600.0,1543.0,...,1780,60,97.0,37.0,17,16,14280,180.0,180.0,120.0


In [27]:
# Делаем датафрейм доступным для других файло ipynb в корневой папке
%store data

Stored 'data' (DataFrame)
